In [ ]:
from seleniumwire import webdriver
import time
import pathlib
import os
import requests
from urllib.parse import urlparse, parse_qs
from io import BytesIO
import pandas as pd
import json


# Get metadata of all bike counting stations of a certain city

In [ ]:
#found by googling https://data.eco-counter.com/ParcPublic/?id=
ids = {
    'Stuttgart': 607,
    'Rostock': 888, 
    'Berlin': 4728, 
    'Bochum': 6603,
    'Düsseldorf': 857,
    'Köln': 677,
    'Reutlingen': 7581
}

In [ ]:
dfs = []

In [ ]:
for k,v in ids.items():
    city = k
    url = f"https://data.eco-counter.com/ParcPublic/?id={v}#"
    # get driver path
    current_path = pathlib.Path().resolve()
    driver_path = os.path.join(current_path, "chromedriver")

    driver = webdriver.Chrome(driver_path)

    driver.get(url)

    # wait for page to load competletely
    time.sleep(1)
    # get all requests made by the page
    all_requests = driver.requests
    
    # define target url, which shall be filtered out
    TARGET_URL = "www.eco-visio.net"
    GET_REQUEST = ""
    
    for request in all_requests:
        # parse url and filter out query param
        parsed_url = urlparse(request.url)
        query_dictionary = parse_qs(parsed_url.query)
        # check if target url matches
        if (parsed_url.netloc == TARGET_URL):
            GET_REQUEST = request
            print(GET_REQUEST.url)
            if GET_REQUEST:
#                break
            
                url_endpoint = GET_REQUEST
                response = requests.get(url_endpoint)
                response

                data_json = json.loads(response.content)
                df_big = pd.DataFrame(data_json)
                dfs.append(df_big)

In [ ]:
df = pd.concat(dfs) 

In [ ]:
df_big = df.copy()

In [ ]:
df = df_big.loc[(df_big['lon'] < 16) & (df_big['lon'] > 6) & (df_big['lat'] < 55) & (df_big['lat'] > 47)][['id_pdc_img', 'idPdc','lienPublic', 'nom', 'lat', 'lon', 'total', 'nomOrganisme']].copy()

In [ ]:
df

In [ ]:
df.to_csv('temp.csv')

In [ ]:
URL_STUTTGART = 'https://data.eco-counter.com/ParcPublic/?id=607#'

In [ ]:
# get driver path
current_path = pathlib.Path().resolve()
driver_path = os.path.join(current_path, "chromedriver")

driver = webdriver.Chrome(driver_path)

driver.get(URL_STUTTGART)

# wait for page to load competletely
time.sleep(1)

In [ ]:
# get all requests made by the page
all_requests = driver.requests

In [ ]:
# define target url, which shall be filtered out
TARGET_URL = "www.eco-visio.net"
GET_REQUEST = ""

Filter out the get request made to the database and get the metadata of all bike counting stations in a city

In [ ]:
for request in all_requests:
    # parse url and filter out query param
    parsed_url = urlparse(request.url)
    query_dictionary = parse_qs(parsed_url.query)
    # check if target url matches
    if (parsed_url.netloc == TARGET_URL):
        GET_REQUEST = request
        print(GET_REQUEST.url)
        break

Fetch metadata information of all counting stations of a certain city

In [ ]:
url_endpoint = GET_REQUEST
response = requests.get(url_endpoint)
response

extract relevant metadata of the counting stations and save it to json file

In [ ]:
data_json = json.loads(response.content)

In [ ]:
counting_station_list = []
for counting_station in data_json:
    counting_station_dict = {}
    counting_station_dict["id"] = counting_station["idPdc"]
    counting_station_dict["name"] = counting_station["nom"]
    counting_station_dict["latitude"] = counting_station["lat"]
    counting_station_dict["longitude"] = counting_station["lon"]
    counting_station_dict["start_time"] = counting_station["debut"]
    counting_station_list.append(counting_station_dict) 

In [ ]:
data = {}
data['Stuttgart'] = counting_station_list

In [ ]:
data

In [ ]:
with open('metadata/counting_stations_metadata.json', 'w') as outfile:
    json.dump(data, outfile)

# Get data from bike counting stations

In [ ]:
df_metadata_counting_stations = pd.read_csv('additional_metadata.csv') 
df_metadata_counting_stations[:5]

In [ ]:
df_metadata_counting_stations['number'] = df_metadata_counting_stations.groupby('name').cumcount() + 1

In [ ]:
df_metadata_counting_stations

In [ ]:
df_row = df_metadata_counting_stations.loc[df_metadata_counting_stations['id_list'] == 100055269]
print(df_row["name"].values[0])
df_row["number"][0]

In [ ]:
# get ids of all stations
counting_station_ids = list(df_metadata_counting_stations["id_list"])

In [ ]:
BEGIN = 20160101
END = 20210930
STEP = 2

In [ ]:
# Create URL for specific bike counting station

def get_data_api_request(counting_station_id):
    driver = webdriver.Chrome(driver_path)

    # define target url, which shall be filtered out
    TARGET_URL = "www.eco-visio.net"
    target_get_request = ""
    BASE_URL = "https://data.eco-counter.com/public2/?id=" +  str(counting_station_id)
    driver.get(BASE_URL)
    print(BASE_URL)
    # wait for page to load competletely
    time.sleep(1)

    # get all requests made by the page
    all_requests = driver.requests
    
    for request in all_requests:
        # parse url and filter out query param
        parsed_url = urlparse(request.url)
        query_dictionary = parse_qs(parsed_url.query)
        # check if target url matches and has a token in query
        if (parsed_url.netloc == TARGET_URL and "t" in query_dictionary):
            target_get_request = request
            break
            
            
    driver.close()
    return target_get_request

# Make Request and fetch the data
    
def create_data_api_request(get_request):
    parsed_url = urlparse(get_request.url)
    query_dictionary = parse_qs(parsed_url.query)
    
    query_dictionary = parse_qs(parsed_url.query)
    
    # Define Query Parameters for fetching the Data
    
    query_dictionary["begin"] = BEGIN
    query_dictionary["end"] = END
    query_dictionary["step"] = STEP
    
    url_endpoint = parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path
    return url_endpoint, query_dictionary

# Convert fetched data stream to csv

def save_data_to_csv(data_json, counting_station_id):
    keys = data_json[0].keys()
    date_list = []
    comptage_list = []
    timestamp_list = []   
    columns = ["date", "comptage", "timestamp"]
    for data_object in data_json:
        date_list.append(data_object["date"])
        comptage_list.append(data_object["comptage"])
        timestamp_list.append(data_object["timestamp"])    
    # Calling DataFrame constructor after zipping
    # both lists, with columns specified
    df = pd.DataFrame(list(zip(date_list, comptage_list,timestamp_list)),
                   columns =columns)  
    
    #create name
    df_row = df_metadata_counting_stations.loc[df_metadata_counting_stations['id_list'] == counting_station_id]  
    file_name = df_row["name"].values[0] + "_" + str(df_row["number"].values[0]) + ".csv"
    
    path = os.path.join("data",file_name )
    
    df.to_csv(path)

In [ ]:
get_request = get_data_api_request(100030582)

In [ ]:
found_data_list = []

for counting_station_id in counting_station_ids:
    
    get_request = get_data_api_request(counting_station_id)
    if get_request != "":
        
        url_endpoint, query_dictionary = create_data_api_request(get_request)
        response = requests.get(url_endpoint, params=query_dictionary)
        print(response)
        
        data_json = json.loads(response.content)
        save_data_to_csv(data_json, counting_station_id)
        
        found_data_list.append(1)
    else:
        
        found_data_list.append(0)        
    
    

In [ ]:
len(found_data_list)

In [ ]:
df_metadata_counting_stations['fetched data ?'] = found_data_list
  



In [ ]:
df_metadata_counting_stations

# Berlin special case (outdated)

In [ ]:
#url_endpoint = GET_REQUEST

BERLIN_URL = "https://www.eco-visio.net/api/aladdin/1.0.0/pbl/publicwebpageplus/4728?withNull=true"
url_endpoint = BERLIN_URL
response = requests.get(url_endpoint)
response


In [ ]:

data_json = json.loads(response.content)
df_berlin = pd.DataFrame(data_json)

In [ ]:
#weird column ordering below to get the existing format - sorry
df = df_berlin[['idPdc', 'lat', 'lon', 'nom']].copy()

In [ ]:
df['name'] = 'Berlin - ' + df['nom'] 

In [ ]:
test = df[['name', 'lat', 'lon', 'nom', 'idPdc']].reset_index().copy()

In [ ]:
test.columns = ['number', 'name', 'lat', 'lon', 'nom', 'id_list']

In [ ]:
df = test[['name', 'number', 'lat', 'lon', 'id_list']]

In [ ]:
df['number'] = df['number'].astype(str)

In [ ]:
df_metadata_counting_stations = df

In [ ]:
# get ids of all stations
counting_station_ids = list(df_metadata_counting_stations["id_list"])
BEGIN = 20160101
END = 20210926
STEP = 2

found_data_list = []

for counting_station_id in counting_station_ids:
    
    get_request = get_data_api_request(counting_station_id)
    if get_request != "":
        
        url_endpoint, query_dictionary = create_data_api_request(get_request)
        response = requests.get(url_endpoint, params=query_dictionary)
        print(response)
        
        data_json = json.loads(response.content)
        save_data_to_csv(data_json, counting_station_id)
        
        found_data_list.append(1)
    else:
        
        found_data_list.append(0)        
    
    

In [ ]:


len(found_data_list)

df_metadata_counting_stations['fetched data ?'] = found_data_list
  





In [ ]:
df

In [ ]:
df.to_csv('berlin_stations_metadata.csv')